In [1]:
import polars as pl
import os

In [2]:
# load the data as a lazy dataframe
data = pl.scan_csv(r'data\downloaded datasets\retailrocket\events.csv',
                   with_column_names=lambda names: ['Time', 'UserId', 'Type', 'ItemId', 'transactionid'])

print(data.fetch(5))
print(data.collect().shape)

shape: (5, 5)
┌───────────────┬────────┬──────┬────────┬───────────────┐
│ Time          ┆ UserId ┆ Type ┆ ItemId ┆ transactionid │
│ ---           ┆ ---    ┆ ---  ┆ ---    ┆ ---           │
│ i64           ┆ i64    ┆ str  ┆ i64    ┆ str           │
╞═══════════════╪════════╪══════╪════════╪═══════════════╡
│ 1433221332117 ┆ 257597 ┆ view ┆ 355908 ┆ null          │
│ 1433224214164 ┆ 992329 ┆ view ┆ 248676 ┆ null          │
│ 1433221999827 ┆ 111016 ┆ view ┆ 318965 ┆ null          │
│ 1433221955914 ┆ 483717 ┆ view ┆ 253185 ┆ null          │
│ 1433221337106 ┆ 951259 ┆ view ┆ 367447 ┆ null          │
└───────────────┴────────┴──────┴────────┴───────────────┘
(2756101, 5)


In [3]:
# remove transactionid colum
data = data.drop('transactionid')

In [4]:
# check for missing values
data.null_count().collect()

Time,UserId,Type,ItemId
u32,u32,u32,u32
0,0,0,0


Create session ids. each session has a maximum lenght of 30 minutes.

In [5]:
# create a timestamp column 
data = (data
          .with_columns(
              pl.from_epoch("Time", time_unit="ms")
              )
          )

data.collect()

Time,UserId,Type,ItemId
datetime[ms],i64,str,i64
2015-06-02 05:02:12.117,257597,"""view""",355908
2015-06-02 05:50:14.164,992329,"""view""",248676
2015-06-02 05:13:19.827,111016,"""view""",318965
2015-06-02 05:12:35.914,483717,"""view""",253185
2015-06-02 05:02:17.106,951259,"""view""",367447
…,…,…,…
2015-08-01 03:13:05.939,591435,"""view""",261427
2015-08-01 03:30:13.142,762376,"""view""",115946
2015-08-01 02:57:00.527,1251746,"""view""",78144


create session ids. Each session has a maximum lenght of 30 minutes.

In [6]:
# sort the dataset on userid and timestamp
data = data.sort(['UserId', 'Time'])

data = (data
       # create two new columns that contain boolean values if the previous row is another user or exceeds the maximum session length
       .with_columns([
           # check if the user_id is different from the previous row
           (pl.col("UserId").diff() != 0).fill_null(True).alias("char_diff"),
           # check if the time difference between the current and previous row exceeds the threshold
           (pl.col("Time").diff().dt.total_minutes().cast(pl.Int64) > 30).fill_null(True).alias("ts_diff"),
       ])

       # create a new column that determines if a new session needs to be started after this row
       .with_columns([
           (pl.col("ts_diff") | pl.col("char_diff")).alias("new_session_mark")
       ])

       # drop the columns that we don't need anymore
       .drop(["char_diff", "ts_diff"])

       # create a new column that contains the session id
       .with_columns([
           (pl.col("new_session_mark").cum_sum()).alias("SessionId")
       ])
       # drop the column that we don't need anymore
       .drop("new_session_mark")
       )
data.collect()

Time,UserId,Type,ItemId,SessionId
datetime[ms],i64,str,i64,u32
2015-09-11 20:49:49.439,0,"""view""",285930,1
2015-09-11 20:52:39.591,0,"""view""",357564,1
2015-09-11 20:55:17.175,0,"""view""",67045,1
2015-08-13 17:46:06.444,1,"""view""",72028,2
2015-08-07 17:51:44.567,2,"""view""",325215,3
…,…,…,…,…
2015-06-10 21:46:08.922,1407575,"""view""",121220,1759829
2015-06-03 15:01:29.991,1407576,"""view""",356208,1759830
2015-05-17 21:48:04.867,1407577,"""view""",427784,1759831


Filter out session ids with only one interaction and also session ids that have items that appear les than 5 times. (item cold start)

In [7]:
# Get session ids with more than one interaction in session
SessionIdsWithMoreThanOneInteraction = (data
                                        # get the length of each session
                                        .group_by('SessionId').len()
                                        # filter out the sessions that only contain one action
                                        .filter(pl.col('len') > 1)
                                        .select('SessionId')
                                        ).collect()

SessionIdsWithMoreThanOneInteraction

SessionId
u32
376990
1037242
1043810
637446
917841
…
368329
319424
1277896


In [8]:
ItemIdsColdstartItems = (data
                         .group_by(['ItemId', 'Type']).len()
                         .filter(pl.col('Type') == 'view')
                         .filter(pl.col('len') < 5)
                         .select('ItemId')
                         ).collect()

print(f'ItemIdsColdstartItems: {ItemIdsColdstartItems}')

SessionIdsWithColdstartItems = (data
                                .filter(pl.col('ItemId').is_in(ItemIdsColdstartItems))
                                .select('SessionId')
                                ).unique().collect()
print(f'SessionIdsWithColdstartItems: {SessionIdsWithColdstartItems}')

ItemIdsColdstartItems: shape: (144_690, 1)
┌────────┐
│ ItemId │
│ ---    │
│ i64    │
╞════════╡
│ 272463 │
│ 8163   │
│ 322109 │
│ 91688  │
│ 268918 │
│ …      │
│ 233592 │
│ 291044 │
│ 208357 │
│ 60273  │
│ 219284 │
└────────┘
SessionIdsWithColdstartItems: shape: (228_342, 1)
┌───────────┐
│ SessionId │
│ ---       │
│ u32       │
╞═══════════╡
│ 247368    │
│ 62541     │
│ 270701    │
│ 1184713   │
│ 1647821   │
│ …         │
│ 55577     │
│ 247719    │
│ 1200019   │
│ 1608735   │
│ 778864    │
└───────────┘


In [9]:
# keep the session ids that have more than one interaction and do not contain coldstart items
data = (data
        .filter(pl.col('SessionId').is_in(SessionIdsWithMoreThanOneInteraction))
        # .filter(~pl.col('ItemId').is_in(ItemIdsColdstartItems))
        .filter(~pl.col('SessionId').is_in(SessionIdsWithColdstartItems))

        # sort the dataset on userid and timestamp
        .sort(['SessionId', 'Time'])
        ).collect()

display(data)
data.shape

Time,UserId,Type,ItemId,SessionId
datetime[ms],i64,str,i64,u32
2015-09-11 20:49:49.439,0,"""view""",285930,1
2015-09-11 20:52:39.591,0,"""view""",357564,1
2015-09-11 20:55:17.175,0,"""view""",67045,1
2015-08-07 17:51:44.567,2,"""view""",325215,3
2015-08-07 17:53:33.790,2,"""view""",325215,3
…,…,…,…,…
2015-08-10 18:42:01.481,1407567,"""view""",219086,1759817
2015-08-01 04:12:07.230,1407573,"""view""",363109,1759826
2015-08-01 04:12:45.311,1407573,"""view""",463766,1759826


(1122762, 5)

Create a train and test set.

In [10]:
min_time = data['Time'].min()
max_time = data['Time'].max()

print(f"First date in the dataset: {min_time}")
print(f"Last date in the dataset: {max_time}")

First date in the dataset: 2015-05-03 03:00:04.384000
Last date in the dataset: 2015-09-18 02:59:47.788000


In [11]:
# take last 7 days of data as test set
test = data.filter(pl.col('Time') > max_time - pl.duration(days=7))

test

Time,UserId,Type,ItemId,SessionId
datetime[ms],i64,str,i64,u32
2015-09-11 20:49:49.439,0,"""view""",285930,1
2015-09-11 20:52:39.591,0,"""view""",357564,1
2015-09-11 20:55:17.175,0,"""view""",67045,1
2015-09-15 17:35:31.705,54,"""view""",388096,66
2015-09-15 17:37:45.550,54,"""view""",283115,66
…,…,…,…,…
2015-09-15 15:20:54.557,1407384,"""view""",308740,1759608
2015-09-13 04:36:06.867,1407422,"""view""",354153,1759650
2015-09-13 04:40:29.604,1407422,"""view""",354153,1759650


In [12]:
# remove test data from the dataset
train = data.filter(pl.col('Time') <= max_time - pl.duration(days=7))

train

Time,UserId,Type,ItemId,SessionId
datetime[ms],i64,str,i64,u32
2015-08-07 17:51:44.567,2,"""view""",325215,3
2015-08-07 17:53:33.790,2,"""view""",325215,3
2015-08-07 17:56:52.664,2,"""view""",259884,3
2015-08-07 18:01:08.920,2,"""view""",216305,3
2015-08-07 18:08:25.669,2,"""view""",342816,3
…,…,…,…,…
2015-08-10 18:42:01.481,1407567,"""view""",219086,1759817
2015-08-01 04:12:07.230,1407573,"""view""",363109,1759826
2015-08-01 04:12:45.311,1407573,"""view""",463766,1759826


In [13]:
print(f"Sessions in train set: {train['SessionId'].n_unique()}")

Sessions in train set: 322114


Filter out items that are in the test data but not in the train data. 

In [14]:
ItemIdsInTest = test.select('ItemId').unique()
ItemIdsInTrain = train.select('ItemId').unique()

# create a list with the item ids that are in the test set but not in the training set
ItemIdsInTestbutNotInTrain = ItemIdsInTest.filter(~pl.col('ItemId').is_in(ItemIdsInTrain))

# remove items that are in the test set but not in the training set
test = test.filter(~pl.col('ItemId').is_in(ItemIdsInTestbutNotInTrain))

test.shape

(40771, 5)

prepare the data for the model by converting the Time back to unix time

In [15]:
train = train.with_columns(
    pl.col("Time").dt.epoch(time_unit='s').alias("Time")
)

test = test.with_columns(
    pl.col("Time").dt.epoch(time_unit='s').alias("Time")
)   

save the datasets

In [16]:
# create directory
os.makedirs('data/processed datasets/retailrocket', exist_ok=True)

train.write_csv('data/processed datasets/retailrocket/events_train.csv',
                separator='\t')
test.write_csv('data/processed datasets/retailrocket/events_test.csv',
               separator='\t')

summary statistics

In [17]:
# concat the train and test data to calculate the statistics
data = pl.concat([train, test])

In [18]:
data.group_by('Type').len()

Type,len
str,u32
"""view""",1051512
"""transaction""",16808
"""addtocart""",52281


In [19]:
data.shape

(1120601, 5)

In [20]:
def summary_stats(df, dataset_str):
    print(f'Summary statistics for the {dataset_str} dataset: ')
    n_events = df.shape[0]
    print(f'Number of events: {n_events}')
    # calculate the number of unique items
    n_items = df.select('ItemId').n_unique()
    print(f'Number of unique items: {n_items}')
    # calculate the number of sessions
    n_sessions = df.select('SessionId').n_unique()
    print(f'Number of sessions: {n_sessions}')
    # calculate the session length
    session_length = df.group_by('SessionId').agg(pl.count('SessionId').alias('len')).select(pl.col('len').mean())[0, 0]
    print(f'Average session length: {round(session_length, 2)}\n')
    
summary_stats(data, 'whole')
summary_stats(train, 'train')
summary_stats(test, 'test')

Summary statistics for the whole dataset: 
Number of events: 1120601
Number of unique items: 75389
Number of sessions: 335175
Average session length: 3.34

Summary statistics for the train dataset: 
Number of events: 1079830
Number of unique items: 75389
Number of sessions: 322114
Average session length: 3.35

Summary statistics for the test dataset: 
Number of events: 40771
Number of unique items: 13857
Number of sessions: 13074
Average session length: 3.12

